In [1]:
import pickle
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

# ======================
# 1. 读取数据
# ======================

fit_path = "PKL_DATA/fitdata/fitresults_ALL_larbi_leaky.pkl"
subj_path = "PKL_DATA/ALL_SUBJECTS.pkl"

# 读取拟合结果（list of dict）
with open(fit_path, "rb") as f:
    fit_list = pickle.load(f)

# 读取被试信息（dict: subj_id -> DataFrame）
with open(subj_path, "rb") as f:
    subj_dict = pickle.load(f)

# ======================
# 2. 构建参数表
# ======================
subj_ids = list(subj_dict.keys())  # 拟合时的被试顺序
param_names = fit_list[0]["param_name"]  # larbi_mini 模型的所有参数

rows = []
for sid, fr in zip(subj_ids, fit_list):
    # 每一行 = 一个被试
    row = {"subj": sid}
    row["group"] = "patient" if "Q" in sid.upper() else "healthy"
    for name, val in zip(param_names, fr["param"]):
        row[name] = float(val)
    rows.append(row)

df = pd.DataFrame(rows)

# 保存一个干净的参数表，方便检查
df.to_csv("larbi_leaky_params_with_groups.csv", index=False)
print(f"✅ 保存参数表: larbi_leaky_params_with_groups.csv")

# ======================
# 3. Welch's t-test
# ======================
results = []
healthy_df = df[df["group"] == "healthy"]
patient_df = df[df["group"] == "patient"]

for param in param_names:
    x = healthy_df[param].dropna().values
    y = patient_df[param].dropna().values

    t_stat, p_val = ttest_ind(x, y, equal_var=False)  # Welch's t-test

    results.append({
        "parameter": param,
        "healthy_mean": np.mean(x),
        "patient_mean": np.mean(y),
        "t_stat": t_stat,
        "p_value": p_val,
        "n_healthy": len(x),
        "n_patient": len(y)
    })

# 转换为 DataFrame 并按 p 值排序
df_ttest = pd.DataFrame(results).sort_values("p_value").reset_index(drop=True)

# 保存结果表
df_ttest.to_csv("t_test_results_larbi_leaky.csv", index=False)
print(f"✅ 保存t检验结果: t_test_results_larbi_leaky.csv")

# 显示结果
print("\n=== Welch's t-test Results ===")
print(df_ttest)


✅ 保存参数表: larbi_leaky_params_with_groups.csv
✅ 保存t检验结果: t_test_results_larbi_leaky.csv

=== Welch's t-test Results ===
  parameter  healthy_mean  patient_mean    t_stat   p_value  n_healthy  \
0      leak      3.022206      4.369466 -1.883373  0.062675         46   
1   alpha_u      0.302030      0.256687  0.727754  0.468557         46   
2      beta      0.144532      0.139084  0.265241  0.791435         46   
3       tau      4.094135      4.299669 -0.244056  0.807712         46   
4     theta      0.307660      0.294087  0.196539  0.844611         46   
5     alpha      0.485019      0.476319  0.174465  0.861864         46   

   n_patient  
0         53  
1         53  
2         53  
3         53  
4         53  
5         53  
